# Join features criadas até o momento com o target 

Nesta aula, vamos explorar como realizar operações de **Join** entre DataFrames no PySpark.<br>

## Documentação do PySpark ([link](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html))
```
DataFrame.join(other, on=None, how=None)
Joins with another DataFrame, using the given join expression.
```

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/06 23:06:54 WARN Utils: Your hostname, MacBook-Air-de-Vitor.local, resolves to a loopback address: 127.0.0.1; using 192.168.3.49 instead (on interface en0)
26/02/06 23:06:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 23:06:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/06 23:06:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/06 23:06:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Juntando os datasets criados até o momento 

Vamos carregar os datasets
- target,
- feature_payment2 e
- feature_delivery

In [2]:
base_path = "data/processed"
target_path = f"{base_path}/target"
payment_path = f"{base_path}/feature_payment_part2"
delivery_path = f"{base_path}/feature_delivery"


target = spark.read.parquet(target_path)
payment = spark.read.parquet(payment_path)
delivery = spark.read.parquet(delivery_path)

In [3]:
# Uma boa pratica quando trabalhamos com join 
# é fazer o count dos datasets antes do join

print(target.count(), payment.count(), delivery.count(), sep="\n")

104162
99440
96476


## Juntando target com payment

Foi definido que será feito um INNER JOIN entre target e payment <br>
Só queremos modelar quem pagou

In [4]:
target.printSchema()
payment.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- target: integer (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- count_payment_value: long (nullable = true)
 |-- min_payment_value: double (nullable = true)
 |-- max_payment_value: double (nullable = true)
 |-- avg_payment_value: double (nullable = true)
 |-- sum_payment_value: double (nullable = true)
 |-- stddev_payment_value: double (nullable = true)



In [5]:
df = (
    target
    .join(
        payment,
        on='order_id',
        how='inner'
    )
)

df.count()

99223

## Juntando com delivery

In [6]:
df = (
    df
    .join(
        delivery,
        on="order_id",
        how="left",
    )
)

df.count()

99223

## Join com Colunas de Nomes Diferentes

Muitas vezes, as colunas chave em duas tabelas têm nomes diferentes (ex: `id` e `order_id`). <br>
Nesse caso, não podemos passar apenas uma string simples para o parametro `on`.

In [7]:
data1 = [(1, "A"), (2, "B")]
df1 = spark.createDataFrame(data1, ["order_id", "value"])

data2 = [(1, "A"), (3, "C")]
df2 = spark.createDataFrame(data2, ["id", "value"])

df1.show()
df2.show()

join_df = df1.join(df2, df1.order_id == df2.id, "inner")
join_df.show()

+--------+-----+
|order_id|value|
+--------+-----+
|       1|    A|
|       2|    B|
+--------+-----+

+---+-----+
| id|value|
+---+-----+
|  1|    A|
|  3|    C|
+---+-----+

+--------+-----+---+-----+
|order_id|value| id|value|
+--------+-----+---+-----+
|       1|    A|  1|    A|
+--------+-----+---+-----+



## Impacto do Join no Processamento

Joins são operações custosas em sistemas distribuídos como o Spark. Aqui estão dois pontos críticos:

### 1. Shuffle (Embaralhamento)
Para realizar o join, o Spark precisa garantir que todos os registros com a mesma chave (`order_id`) estejam na mesma partição (na mesma máquina/executor). 
- Isso causa o **Shuffle**: movimentação física de dados pela rede entre os nós do cluster.
- O Shuffle é caro em termos de I/O de disco e rede, e muitas vezes é o gargalo de performance.

### 2. Skew Join (Distorção)
Se a distribuição das chaves for desigual, ocorre o **Data Skew**.
- Imagine que um único `order_id` tivesse milhões de registros de pagamento (o que não deve acontecer aqui, mas é comum em chaves como "país" ou "categoria").
- A partição que processar essa chave ficará sobrecarregada, demorando muito mais que as outras.
- Isso deixa o cluster ocioso esperando uma única tarefa terminar ("straggler task").

## Salvando o DataFrame

In [8]:
path = "data/processed/join_features"

df.write.mode("overwrite").parquet(path)